# Granule Cutout Training

In [24]:
from ultralytics import YOLO
from PIL import Image
import plotly.express as px
import plotly.graph_objects as go
import imageio.v3 as iio
import numpy as np
from matplotlib import pyplot as plt
from plotly.subplots import make_subplots
import helper_functions as hf
import pandas as pd
from helper_functions import get_border_pixels
from pathlib import Path
import pathlib
import platform
plt2 = platform.system()
if plt2 == 'Windows': pathlib.PosixPath = pathlib.WindowsPath

In [25]:
import torchvision
import torch

print("Cuda:",torch.cuda.is_available())
torchvision.__version__

Cuda: True


'0.16.2+cu118'

# Notes

For prediction, add 'retina_masks=True' for higher quality mask images. [link](https://github.com/ultralytics/ultralytics/issues/2090)

Run in cmd
### yolo segment train data=yolov8-seg-test.yaml model=yolov8_models/yolov8n-seg.pt pretrained=runs/segment/train12/weights/last.pt epochs=10 imgsz=1024

In [26]:

# Load a model
# model = YOLO('yolov8x-seg.yaml')  # build a new model from YAML

# this one
model: YOLO = YOLO('yolov8_models/yolov8n-seg.pt')  # load a pretrained model (recommended for training)

# model = YOLO('yolov8n-seg.yaml').load('yolov8n.pt')  # build from YAML and transfer weights
# model = YOLO('YOLOv8x-seg.yaml').load('YOLOv8x-seg.pt')  # build from YAML and transfer weights


# This one for training
# results = model.train(data='yolov8-seg-test.yaml', epochs=3, imgsz=1024)

In [27]:
# Load a model
# model = YOLO('yolov8n-seg.pt')  # load an official model
# model = YOLO('runs\\segment\\train\\weights\\best.pt')  # load a custom model
model: YOLO = YOLO('runs\\segment\\train13\\weights\\last.pt')  # load a custom model

# Model predicted bounding boxes & masks

In [28]:
def draw_masks_and_bbs():
    # Predict with the model
    frame_id = 493
    granule_id = 15
    image_to_predict = f"datasets/granule-seg_cutout/images/val/2020-02-05_15.41.32-NAs-T1354-GFP_Burst_Frame_{frame_id}_Granule_{granule_id}.png"
    # image_to_predict = f"Not_valid_granule.png"
    results: YOLO = model(image_to_predict, verbose=False)  # predict on an image

    # Show the results
    for r in results:
        im_array = r.plot()  # plot a BGR numpy array of predictions
        
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segmentation masks outputs
        probs = r.probs  # Class probabilities for classification outputs

        im = iio.imread(image_to_predict)
        fig = px.imshow(im)

        for mask, box in zip(r.masks.xy, r.boxes):
            points = np.int32(mask)
            
            # BB
            bb = box.xyxy[0]
            fig.add_shape(type='rect', 
                        x1=bb[0],y1=bb[1], x0=bb[2], y0=bb[3],
                        line=dict(color='red', width=2),
                        fillcolor='rgba(255, 0, 0, 0.2)',
                        )
            # Mask
            fig.add_trace(go.Scatter(x=mask[:,0], y=mask[:,1], marker=dict(color='cyan', size=4), fill='toself'))

        fig.show()
# draw_masks_and_bbs()

# Model outputs

In [29]:
analyzed_granule_results = pd.read_hdf(Path("datasets/analyzed_granule_data/2020-02-05_15.41.32-NAs-T1354-GFP_Burst.h5"), mode="r", key="fourier")
def make_plot(analyzed_granule_results, results, image_to_predict, frame_id, granule_id, draw_true_labels=True):
    fig = make_subplots(
        rows=2, cols=2, horizontal_spacing=0.01, vertical_spacing=0.05,subplot_titles=('Base image + True labels','Full prediction', 'Predicted Mask + True label', 'Predicted Mask'))
        # specs=[[{"type": "scatter"}, {"type": "bar"}],
            #    [{"type": "image"}, {"type": "surface"}]])
    im = iio.imread(image_to_predict)
    base_image = px.imshow(im)

    result_masks = results[0].plot(conf=False, line_width=0, font_size=0,img=np.zeros((1024, 1024, 3), dtype=np.uint8), kpt_radius=0, kpt_line=False, labels=False, boxes=False, masks=True, probs=False)
    result_full = results[0].plot()
    image_result_masks = px.imshow(result_masks)
    image_result_full = px.imshow(result_full)
    fig.add_trace(base_image.data[0], 1, 1)
    fig.add_trace(image_result_full.data[0], 1, 2)
    fig.add_trace(image_result_masks.data[0], 2, 1)
    fig.add_trace(image_result_masks.data[0], 2, 2)


    # Add Valid granule borders
    granule_fourier2 = analyzed_granule_results[(analyzed_granule_results['granule_id'] == granule_id) & (analyzed_granule_results['frame'] == frame_id)]
    xs, ys = hf.get_coords(granule_fourier2, get_relative=True)
    # ---------- Get scale factors -----------
    BB_BOX = granule_fourier2.iloc[0]
    bbox_left = granule_fourier2['bbox_left'].iloc[0]
    bbox_right = granule_fourier2['bbox_right'].iloc[0]
    bbox_top = granule_fourier2['bbox_top'].iloc[0]
    bbox_bottom = granule_fourier2['bbox_bottom'].iloc[0]
    original_width, original_height = (abs(bbox_right - bbox_left), abs(bbox_bottom - bbox_top))
    scale_factor_height = 1024 / original_height
    scale_factor_width = 1024 / original_width
    phi_x = scale_factor_width / 2 - 1/2 # Add offset due to up-resizing
    phi_y = scale_factor_height / 2 - 1/2
    # ---------- Line segements coords ----------
    ys_coords = np.round(ys, 0)
    xs_coords = np.round(xs, 0)
    if draw_true_labels:
        fig.add_trace(go.Scatter(y=phi_y+ys*scale_factor_width, x=phi_x+xs*scale_factor_height, marker=dict(color='red', size=16), name=f"True Border - Granule {granule_id}"), row=1, col=1)
        fig.add_trace(go.Scatter(y=phi_y+ys*scale_factor_width, x=phi_x+xs*scale_factor_height, marker=dict(color='red', size=16), name=f"True Border - Granule {granule_id}"), row=2, col=1)
    # fig.add_trace(go.Scatter(y=ys_coords, x=xs_coords, marker=dict(color='cyan', size=8), fill='toself', name=f"Pixel Border - Granule {granule_id}"), col=1, row=1)

    fig.update_layout(title_text=f"Model output - Frame {frame_id} Granule {granule_id}", title_x=0.5, height=1200, width=1200, showlegend=False, font_size=11)
    fig.show()
    # fig.add_image(new)
    # mask_img = result.plot(conf=False, line_width=0, font_size=0, img=np.zeros((img_height, img_width, 3), dtype=np.uint8), kpt_radius=0, kpt_line=False, labels=False, boxes=False, masks=True, probs=False)

## Model output to fourier terms

In [47]:
frame_id = 493
granule_id = 15
image_to_predict = f"datasets/granule-seg_cutout/images/val/2020-02-05_15.41.32-NAs-T1354-GFP_Burst_Frame_{frame_id}_Granule_{granule_id}.png"
granule_fourier = analyzed_granule_results[(analyzed_granule_results['granule_id'] == granule_id) & (analyzed_granule_results['frame'] == frame_id)]
results: YOLO = model(image_to_predict, verbose=False)  # predict on an image

x_center, y_center = granule_fourier.iloc[0][['x','y']]

border_image = np.zeros((1024,1024,3)) 
for x,y in get_border_pixels(results):
    border_image[x][y] = [100,100,100]
image_result_masks = px.imshow(border_image)
image_result_masks.add_trace(go.Scatter(y=[x_center,x_center], x=[y_center,y_center] , marker=dict(color='red', size=16), name=f"Granule center"), row=1, col=1)
image_result_masks.show()



# Valid granule examples

In [ ]:
# Predict with the model
frame_id = 4
granule_id = 7
image_to_predict = f"datasets/granule-seg_cutout/images/val/2020-02-05_15.41.32-NAs-T1354-GFP_Burst_Frame_{frame_id}_Granule_{granule_id}.png"
results = model(image_to_predict, verbose=False)  # predict on an image
make_plot(analyzed_granule_results, results, image_to_predict, frame_id, granule_id)

In [ ]:
# Predict with the model
frame_id = 0
granule_id = 4
image_to_predict = f"datasets/granule-seg_cutout/images/train/2020-02-05_15.41.32-NAs-T1354-GFP_Burst_Frame_{frame_id}_Granule_{granule_id}.png"
results = model(image_to_predict, verbose=False)  # predict on an image
make_plot(analyzed_granule_results, results, image_to_predict, frame_id, granule_id)

In [ ]:
# Predict with the model
frame_id = 183
granule_id = 120
image_to_predict = f"datasets/granule-seg_cutout/images/val/2020-02-05_15.41.32-NAs-T1354-GFP_Burst_Frame_{frame_id}_Granule_{granule_id}.png"
results = model(image_to_predict, verbose=False)  # predict on an image
make_plot(analyzed_granule_results, results, image_to_predict, frame_id, granule_id)

In [ ]:
frame_id = 493
granule_id = 15
image_to_predict = f"datasets/granule-seg_cutout/images/val/2020-02-05_15.41.32-NAs-T1354-GFP_Burst_Frame_{frame_id}_Granule_{granule_id}.png"
# image_to_predict = f"Not_valid_granule.png"
results = model(image_to_predict, verbose=False)  # predict on an image
make_plot(analyzed_granule_results, results, image_to_predict, frame_id, granule_id)

# Examples of not valid granules

In [ ]:
frame_id = 0
granule_id = 0
image_to_predict = f"images/Not_valid_granule.png"
results = model(image_to_predict, verbose=False)  # predict on an image
make_plot(analyzed_granule_results, results, image_to_predict, frame_id, granule_id, draw_true_labels=False)

In [ ]:
frame_id = 0
granule_id = 6
image_to_predict = f"../dataset_creation/datasets/not_valid_granule_cutouts/2020-02-05_15.41.32-NAs-T1354-GFP_Burst_Frame_{frame_id}_Granule_{granule_id}.png"
results = model(image_to_predict, verbose=False)  # predict on an image
make_plot(analyzed_granule_results, results, image_to_predict, frame_id, granule_id, draw_true_labels=False)

In [ ]:
frame_id = 0
granule_id = 30
image_to_predict = f"../dataset_creation/datasets/not_valid_granule_cutouts/2020-02-05_15.41.32-NAs-T1354-GFP_Burst_Frame_{frame_id}_Granule_{granule_id}.png"
results = model(image_to_predict, verbose=False)  # predict on an image
make_plot(analyzed_granule_results, results, image_to_predict, frame_id, granule_id, draw_true_labels=False)

In [ ]:
frame_id = 0
granule_id = 14
image_to_predict = f"../dataset_creation/datasets/not_valid_granule_cutouts/2020-02-05_15.41.32-NAs-T1354-GFP_Burst_Frame_{frame_id}_Granule_{granule_id}.png"
results = model(image_to_predict, verbose=False)  # predict on an image
make_plot(analyzed_granule_results, results, image_to_predict, frame_id, granule_id, draw_true_labels=False)

# Comments/thoughts



### Dataset
Training data only consists of granules we already know how to identify. -> Need to find a way to teach the model to recognize new granules.

    - Modify Granule Explorer to give estimates for non-circular granule's?
    - Create synthetic data? Simulate granule shapes and create custom data samples?

    - Note: This test model is only trained on one experiment. Since all frames are fairly similar, model accuracy are most likely lower. 

### Model's and model size
Currently only able to use smallest YOLOv8 model. (The 'nano' version)

    - If more computing power is available -> Train larger model. Might be able to pick up on more complex underlying data structures for better precision.

    - Test more model architectures. YOLOv8 works, can also do 
        - Segment Anything Model
        - Detectron2 Mask-RCNN
        - YOLO-NAS

### Switch strategy? (Problem with output accuracy, what do we want to achieve?)
The full microscope images are large with many small 'features'. Granules are only 20-40 pixel wide/high, which is difficult for networks to work with.
Instead, crop out all potential granules and classify them only? Skipping all the background.

    - Can then also upscale each granule by 10x, giving us more accuracy.






Sper ocmputer uib/uio -> nrec